<h2>Segmenting and Clustering Neighborhoods in Toronto</h2>

Import the Libraries we are going to use

In [2]:
import requests
from lxml import etree
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page = requests.get(wikipedia_link)
tree = etree.HTML(raw_wikipedia_page.text)

<ul>
    <li>Loop the table</li>
    <li>Assign Borough as Neighborhood when Neighborhood is Not assigned</li>
    <li>Concatenate Neighborhoods that have the same PostalCode</li>
</ul>

In [5]:
import pandas as pd

labels = ['Postal Code', 'Borough', 'Neighborhood']
listDict = {}
#Iterating the table
for table in tree.xpath('//table[@class="wikitable sortable"]'):
    for row in table.xpath('tbody/tr'):
        cols = row.xpath('td//text()')
        #Condition to skip the rows with Borough as Not assigned
        if len(cols)>0 and cols[1]!='Not assigned':
            try:
                cols.remove('\n')
            except:
                pass 
            cols[2] = cols[2].rstrip()
            #Assigning Borough as Neighborhood when Neighborhood is Not assigned
            if(cols[2]=='Not assigned'):
                cols[2] = cols[1]
            #Concatenating Neighborhoods that have the same PostalCode    
            if(cols[0] in listDict):
                value = listDict.get(cols[0])
                neighborhoodList = value[2]+', '+cols[2]
                value[2] = neighborhoodList
                listDict[cols[0]] = value
            else:
                listDict[cols[0]] = cols
 
valList = [ v for v in listDict.values() ]
df = pd.DataFrame.from_records(valList, columns=labels)
df.head()


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Display the shape of the dataframe

In [6]:
df.shape

(103, 3)

Reading the latitude and longitude info from the csv file

In [7]:
df_loc = pd.read_csv('Geospatial_Coordinates.csv')

In [8]:
df_loc.tail()

,Postal Code,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


Merge the Longitude and Latitude into the dataframe

In [9]:
 final_df = pd.merge(df, df_loc, on='Postal Code')

In [10]:
final_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


Display the number of Boroughs and Neighborhoods

In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(final_df['Borough'].unique()),
        final_df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


Filter the data to get only the Boroughs that contain the word Toronto

In [14]:
toronto_data = final_df[final_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Print the Boroughs that contain the word Toronto

In [16]:
toronto_data['Borough'].unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

The number of Boroughs and neighborhoods after filtering data

In [17]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighborhoods.


Let's get the eographical locaion of Toronto

In [21]:
#from geopy.geocoders import Nominatim
#address = 'Toronto, Ontario'

#geolocator = Nominatim()
#location = geolocator.geocode(address)
#latitude = location.latitude
#longitude = location.longitude
latitude = 43.6532
longitude = 79.3832

print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6532, 79.3832.


Create a map of Toronto with neighborhoods superimposed on top.

Import library

In [24]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    branca-0.3.0               |             py_0          24 KB  conda-forge
    altair-2.2.2               |        py36_1001         494 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.3 MB

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py36_1001   conda-forge
    branca:  0.3.0-py_0  

In [25]:
import folium
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare credentials

In [26]:
CLIENT_ID = '5JI0M55ZPTCAWGLYDKMIV4V11QJDPK3NH4WELPVE33XSMZOU' # your Foursquare ID
CLIENT_SECRET = 'RUCH3DB2XQ5CNHCS33ARYWZHJKGBJ2XZKOLF2DSWX03YZUFB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5JI0M55ZPTCAWGLYDKMIV4V11QJDPK3NH4WELPVE33XSMZOU
CLIENT_SECRET:RUCH3DB2XQ5CNHCS33ARYWZHJKGBJ2XZKOLF2DSWX03YZUFB


<h3> Explore Neighborhoods in Manhattan </h3>

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Calling the function on each neighborhood and create a new dataframe called *toronto_venues*

In [29]:
LIMIT = 100
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

Checking the size of the resuling dataframe

In [30]:
print(toronto_venues.shape)
toronto_venues.head()

(1706, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront, Regent Park",43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center


Venues in each of the neighborhoods

In [31]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business reply mail Processing Centre969 Eastern,15,15,15,15,15,15
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",49,49,49,49,49,49
Central Bay Street,80,80,80,80,80,80
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


Unique categories from the venues

In [32]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 231 uniques categories.


<h2>Analyzing neighborhoods</h2>

In [33]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
toronto_onehot.shape

(1706, 231)

In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.00,0.000000,0.00000,0.00000,0.010000,0.000000,0.000000,0.010000,0.01
1,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
3,Business reply mail Processing Centre969 Eastern,0.066667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.00,0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,...,0.000000,0.00,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.012500,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00000,0.012500,0.000000,0.000000,0.012500,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.01,0.010000,0.00000,0.00000,0.060000,0.000000,0.040000,0.010000,0.00
8,Christie,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.011236,0.00,0.011236,0.011236,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.011236,0.00,0.000000,0.00000,0.00000,0.011236,0.011236,0.011236,0.000000,0.00


In [38]:
toronto_grouped.shape

(38, 231)

Each neighborhood along with the top 5 most common venues

In [40]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2          Café  0.04
3      Beer Bar  0.04
4   Cheese Shop  0.04


----Brockton, Exhibition Place, Parkdale Village----
                venue  freq
0         Coffee Shop  0.14
1      Breakfast Spot  0.09
2                Café  0.09
3  Falafel Restaurant  0.05
4       Burrito Place  0.05


----Business reply mail Processing Centre969 Eastern----
                venue  freq
0         Yoga Studio  0.07
1       Auto Workshop  0.07
2       Garden Center  0.07
3              Garden  0.07
4  Light Rail Station  0.07


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1  Airport Terminal 

Moving the data into a pandas dataframe

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Gym,Restaurant,Cosmetics Shop,Hotel,Bar
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Bakery,Beer Bar,Steakhouse,Restaurant,Farmers Market,Café
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Gym,Italian Restaurant,Office,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym
3,Business reply mail Processing Centre969 Eastern,Yoga Studio,Garden,Smoke Shop,Light Rail Station,Farmers Market,Spa,Fast Food Restaurant,Brewery,Restaurant,Auto Workshop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Terminal,Airport Service,Airport Lounge,Boat or Ferry,Airport Gate,Boutique,Plane,Sculpture Garden,Airport Food Court,Airport
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pizza Place,Café,Bakery,Italian Restaurant,Pub,Indian Restaurant,Park,Pet Store
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Sandwich Place,Bar,Falafel Restaurant,Ice Cream Shop
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Bakery,Mexican Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Coffee Shop,Grocery Store
8,Christie,Grocery Store,Café,Park,Nightclub,Baby Store,Italian Restaurant,Diner,Athletics & Sports,Restaurant,Convenience Store
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Burger Joint,Restaurant,Café,Pub,Men's Store,Mediterranean Restaurant


<h2>Cluster Neighborhoods</h2>
Run k-means to cluster the neighborhoods into 5 clusters

In [45]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Data frame with the cluster and the top 10 venues for each neighborhood

In [46]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Restaurant,Pub,Theater,Mexican Restaurant,Dessert Shop
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bar,Japanese Restaurant,Italian Restaurant,Ramen Restaurant,Sandwich Place,Pizza Place
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Cocktail Bar,Gastropub,Italian Restaurant,Bakery,Cosmetics Shop
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Gym / Fitness Center,Trail,Pub,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Bakery,Beer Bar,Steakhouse,Restaurant,Farmers Market,Café


Visualizing the clusters

In [49]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Restaurant,Pub,Theater,Mexican Restaurant,Dessert Shop
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bar,Japanese Restaurant,Italian Restaurant,Ramen Restaurant,Sandwich Place,Pizza Place
2,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Cocktail Bar,Gastropub,Italian Restaurant,Bakery,Cosmetics Shop
3,East Toronto,0,Coffee Shop,Gym / Fitness Center,Trail,Pub,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
4,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Bakery,Beer Bar,Steakhouse,Restaurant,Farmers Market,Café
5,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Sandwich Place,Bar,Falafel Restaurant,Ice Cream Shop
6,Downtown Toronto,0,Grocery Store,Café,Park,Nightclub,Baby Store,Italian Restaurant,Diner,Athletics & Sports,Restaurant,Convenience Store
7,Downtown Toronto,0,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Gym,Restaurant,Cosmetics Shop,Hotel,Bar
8,West Toronto,0,Supermarket,Bakery,Liquor Store,Furniture / Home Store,Café,Middle Eastern Restaurant,Bank,Bar,Discount Store,Fast Food Restaurant
9,Downtown Toronto,0,Coffee Shop,Hotel,Aquarium,Pizza Place,Café,Scenic Lookout,Brewery,Italian Restaurant,Sports Bar,Sushi Restaurant
